## Import dependencies

In [1]:
%cd ..

/mnt/c/Users/ankit/Desktop/Portfolio/kaggle/drawing-with-llms


In [22]:
from dotenv import load_dotenv
import pandas as pd
from utils.llm import llm
import os
import time
import random
from json import loads
from datasets import load_dataset
from tqdm import tqdm
from utils.process_response import extract_json_response

load_dotenv(".env")

True

## View Descriptions

In [13]:
data = load_dataset("csv", data_files="data/combined_train.csv")["train"]

data["description"]

['a purple forest at dusk', 'gray wool coat with a faux fur collar', 'a lighthouse overlooking the ocean', 'burgundy corduroy pants with patch pockets and silver buttons', 'orange corduroy overalls', 'a purple silk scarf with tassel trim', 'a green lagoon under a cloudy sky', 'crimson rectangles forming a chaotic grid', 'purple pyramids spiraling around a bronze cone', 'magenta trapezoids layered on a transluscent silver sheet', 'a snowy plain', 'black and white checkered pants', 'a starlit night over snow-covered peaks', 'khaki triangles and azure crescents', 'a maroon dodecahedron interwoven with teal threads']


We need more descriptions in order to consider Fine Tuning

Define a loop to randomly sample descriptions from the dataset and generate more similar samples using Gemini 2.5 Flash

In [29]:
# modified prompt from https://github.com/camel-ai/camel/blob/a48b948cbddd80b1cf5ad4d7ef99b3bfc21ec9ec/camel/datasets/few_shot_generator.py#L32

SYSTEM_PROMPT = """**You are an advanced data generation assistant.**  
Your goal is to generate high-quality synthetic data samples based on 
provided examples. Your output must be well-structured, 
logically sound, and formatted correctly. 

**Instructions:**
1. **Follow the Structure**  
   Each data sample must include:  
   - **Description**: A clear, well-formed description of an SVG image.  
   - **Rationale**: A step-by-step, reasoning process for generation of the Description.  

2. **Output Format (Strict)**
   - A list of json consisteing of five data samples
```
[{"description" : "Generated description"
"rationale": "reasoning why this description makes sense as a data sample"},
...
]
```

**Now, generate 5 new data samples based on the given examples.**
"""

total_samples = 10
num_samples_gen = 5
sample_collection = []

model = llm(model="gemini-2.5-flash-preview-04-17", api_key=os.getenv("GEMINI_API_KEY"))

model.messages.append(
    {"role": "system", "content": SYSTEM_PROMPT}
)

for i in tqdm(range(int(total_samples / num_samples_gen))):
    num_examples = random.randrange(5, 10)
    indices = [random.randrange(0, len(data)) for _ in range(num_examples)]

    # every iteration randomly add samples from the dataset in context
    USER_PROMPT = "<examples>" + "\n".join(data[indices]["description"]) + "</examples>"

    response = model(USER_PROMPT)

    try:
        samples = loads(extract_json_response(response))
    except Exception:
        samples = [{"description": ""}] * num_samples_gen

    for sample in samples:
        sample_collection.append(sample["description"])

    # remove user prompt
    model.messages.pop()
    # manual delay for rate limit
    time.sleep(2)

unique_sample_collection = set(sample_collection)

df_save = pd.DataFrame({"description": list(unique_sample_collection)})
df_save.dropna(inplace=True)
# df_save.to_csv("data/descriptions.csv", index=False)
df_save.head()

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:17<00:00,  8.64s/it]


,description
0,Emerald wavy lines intersecting silver straigh...
1,Crimson rhombi interconnected by thin brass li...
2,A beige sphere textured like wood grain
3,Crimson hexagons orbiting a white sphere
4,Teal waves rippling across a sandy plane.
